# Import files

In [1]:
from Strats.PairTradingStrat import *
from Strats.PortfolioMetrics import *
from Strats.WRC import *

import warnings

warnings.filterwarnings("ignore")

# Run Strats

In [2]:
# Get all available tickers
response = requests.get(f"{BASE_URL}/ticker/price")
data = response.json()
BTC_pairs = [i["symbol"] for i in data if "BTC" in i["symbol"]]


In [11]:
coint_pairs = get_coint_pairs(BTC_pairs, interval = '5m', start_date="2024-01-01", end_date="2024-01-31")
len(coint_pairs)

100%|██████████| 524/524 [16:11<00:00,  1.85s/it]


62

In [102]:
test_pairs = coint_pairs[:4]
test_pairs = ['BTCUSDT', 'ETHUSDT']
print(test_pairs)

symbol_manager = BinanceSymbolManager()
# Add symbols
for t in test_pairs:
    print(symbol_manager.add_symbol(t))  # Success

price_fetcher = BinancePriceFetcher(symbol_manager.get_symbols())
# Fetch pair historical price
pair_portfolio = price_fetcher.get_grp_historical_ohlcv(
        interval="5m",
        start_date="2024-11-01",
        end_date="2024-12-31"
    )

pair_portfolio

['BTCUSDT', 'ETHUSDT']
'BTCUSDT' added successfully.
'ETHUSDT' added successfully.


,timestamp,BTCUSDT,ETHUSDT
0,2024-11-01 00:00:00,70349.67,2522.94
1,2024-11-01 00:05:00,70382.00,2524.90
2,2024-11-01 00:10:00,70461.99,2527.19
3,2024-11-01 00:15:00,70453.75,2527.86
4,2024-11-01 00:20:00,70398.96,2525.93
...,...,...,...
17276,2024-12-30 23:40:00,92756.49,3363.79
17277,2024-12-30 23:45:00,92725.99,3364.12
17278,2024-12-30 23:50:00,92789.83,3362.51
17279,2024-12-30 23:55:00,92792.05,3361.84


In [103]:
def hurst_exponent(ts: pd.Series, max_lags: int = 50) -> float:
    """
    Calculate the Hurst Exponent of a time series.
        
    Args:
        time_series (pd.Series): Input time series (e.g., stock prices, spreads).
        max_lags (int): Maximum number of lags to compute rescaled range (R/S).
        
    Returns:
        float: Hurst Exponent value.
    """
    # Create the range of lag values
    lags = range(2, 100)

    # Calculate the array of the variances of the lagged differences
    tau = [np.sqrt(np.std(np.subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    # Use a linear fit to estimate the Hurst Exponent
    poly = np.polyfit(np.log(lags), np.log(tau), 1)

    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0


class pair_trading:
    def __init__(self, df, weights):
        self.df = df
        self.tickers = df.columns.tolist()[1:]
        self.weights = weights


    def generate_signals(self, lookback, execute_threshold, close_threshold):
        # Calculate the rolling mean with a window size of lookback
        for t in self.tickers:
            self.df[t+'_rolling_mean'] = self.df[t].rolling(window=lookback).mean()
            self.df[t+'_rolling_std'] = self.df[t].rolling(window=lookback).std()
            self.df[t+'_z_scores'] = (self.df[t] - self.df[t+'_rolling_mean']) / self.df[t+'_rolling_std']
            is_mean_revert = []
            for i in range(lookback, len(self.df)):
                if hurst_exponent(self.df[t][i-lookback:i].values) < 0.5:
                    is_mean_revert += [1]
                else:
                    is_mean_revert += [0]
            self.df[t+'_is_mean_revert'] = [0] * (len(self.df) - len(is_mean_revert)) + is_mean_revert
            self.df[t+"_is_mean_revert"] = self.df[t+"_is_mean_revert"].fillna(0)

            self.df[t+'_signal'] = np.where((self.df[t+'_z_scores'] > execute_threshold) | (self.df[t+'_z_scores'] < -execute_threshold), 1, 0)
            self.df[t+'_signal'] = np.where(self.df[t+'_is_mean_revert'] != 1, 0, self.df[t+'_signal'])
            # self.df.loc[self.df[t+'_z_scores'] > execute_threshold, t+'_signal'] = -1
            # self.df.loc[self.df[t+'_z_scores'] < -execute_threshold, t+'_signal'] = 1

            self.df[t+'_exit_signal'] = np.where((self.df[t+'_z_scores'] > -close_threshold) & (self.df[t+'_z_scores'] < close_threshold), 1, 0)
            self.df[t+'_exit_signal'] = np.where(self.df[t+'_exit_signal'] != 1, 1, self.df[t+'_exit_signal'])

        return self.df

    def computePnL(self, ticker, test_start_date): #have not account for capital
        trades = self.df[self.df['timestamp'] >= test_start_date].copy().reset_index(drop=True)
        # PnL variables - one set per security
        position = 0
        pnlUnrealized = 0
        pnlRealized = 0

        avg_short_price = 0
        short_pos = 0
        avg_long_price = 0
        long_pos = 0
        closed_pos = 0

        positions = []
        pnlUnrealized_list = []
        pnlRealized_list = []
        daily_pnl = []
        daily_pnl_pct = []

        # for each trade
        for i in range(0, len(trades)):
            qty = trades[ticker+'_signal'][i]
            price = trades[ticker][i]
            exit_signal = trades[ticker+'_exit_signal'][i]

            if exit_signal == 1:
                if position != 0:
                    qty = -position #close position

            if qty < 0:
                avg_short_price = (avg_short_price * short_pos + price * qty) / (short_pos + qty)
                short_pos += qty
            elif qty > 0:
                avg_long_price = (avg_long_price * long_pos + price * qty) / (long_pos + qty)
                long_pos += qty

            if i > 0:
                if (qty * position) < 0:
                    closed_pos = min(abs(qty), abs(position))
                else:
                    closed_pos = 0
                short_pos += closed_pos
                long_pos -= closed_pos

                if (position+qty) < 0:
                    pnlUnrealized = (avg_short_price - price) * -(position+qty)
                else:
                    pnlUnrealized = (avg_long_price - price) * (position+qty)
                # print(closed_pos)

            position += qty
            pnlRealized += (avg_short_price - avg_long_price) * closed_pos
            daily_pnl += [(avg_short_price - avg_long_price) * closed_pos + pnlUnrealized]

            positions += [position]
            pnlUnrealized_list += [pnlUnrealized]
            pnlRealized_list += [pnlRealized]
            try:
                daily_pnl_pct += [((pnlRealized+pnlUnrealized) - (pnlUnrealized_list[-2] + pnlRealized_list[-2]))/(pnlUnrealized_list[-2] + pnlRealized_list[-2]+1e-10)]
            except:
                daily_pnl_pct += [np.nan]

            if short_pos == 0:
                avg_short_price = 0
            if long_pos == 0:
                avg_long_price = 0

        pnl_df = trades[["timestamp", ticker, ticker+"_signal"]].copy()
        pnl_df.columns = ["Date", "Price", "Signal"]
        pnl_df["Positions"] = positions
        pnl_df["Realized_PnL"] = pnlRealized_list
        pnl_df["Unrealized_PnL"] = pnlUnrealized_list
        pnl_df["Daily_PnL"] = daily_pnl
        pnl_df["Daily_PnL_Pct"] = daily_pnl_pct
        pnl_df["PnL_Total"] = pnl_df["Realized_PnL"] + pnl_df["Unrealized_PnL"]

        return pnl_df

    def computePortfolioPnL(self, test_start_date):
        portfolioPnL = pd.DataFrame()
        for ind, t in enumerate(self.tickers):
            pnl_df = self.computePnL(t, test_start_date)
            portfolioPnL[t+"_daily_pnl"] = pnl_df["Daily_PnL"]
            portfolioPnL[t+"_daily_pnl_pct"] = pnl_df["Daily_PnL_Pct"]
            if ind > 0:
                portfolioPnL["total_daily_pnl"] = portfolioPnL["total_daily_pnl"] + portfolioPnL[t+"_daily_pnl"] * self.weights[ind]
                portfolioPnL["total_daily_pnl_pct"] = portfolioPnL["total_daily_pnl_pct"] + portfolioPnL[t+"_daily_pnl_pct"] * self.weights[ind]
                portfolioPnL["total_pnl"] = portfolioPnL["total_pnl"] + pnl_df["PnL_Total"] * self.weights[ind]
            else:
                portfolioPnL["total_daily_pnl"] = portfolioPnL[t+"_daily_pnl"] * self.weights[ind]
                portfolioPnL["total_daily_pnl_pct"] = portfolioPnL[t+"_daily_pnl_pct"] * self.weights[ind]
                portfolioPnL["total_pnl"] = pnl_df["PnL_Total"] * self.weights[ind]

        portfolioPnL["timestamp"] = pnl_df["Date"]
        portfolioPnL = portfolioPnL[["timestamp", "total_daily_pnl", "total_daily_pnl_pct", "total_pnl"]]

        return portfolioPnL
    
    def plot_pnl(self, test_start_date):
        fig, axs = plt.subplots(len(self.tickers)+1,1, figsize=(15,9), sharex=True)

        for i, t in enumerate(self.tickers):
            pnl_df = self.computePnL(t, test_start_date)
            axs[i].plot(pnl_df["Date"], pnl_df["PnL_Total"], label=f"{t}_PnL_Total")
            axs[i].set_ylabel("Accumulated PnL")
            axs[i].set_xlabel("Date")
            axs[i].legend()
            axs[i].grid(True)

        total_pnl_df = self.computePortfolioPnL(test_start_date)
        axs[len(self.tickers)].plot(total_pnl_df["timestamp"], total_pnl_df["total_pnl"], label="Portfolio")
        axs[len(self.tickers)].set_ylabel("Accumulated PnL")
        axs[len(self.tickers)].set_xlabel("Date")
        axs[len(self.tickers)].legend()
        axs[len(self.tickers)].grid(True)

        plt.show()

In [ ]:
weights = np.array([1/len(test_pairs) for i in range(len(test_pairs))])
model = pair_trading(pair_portfolio.copy(), weights)
spread = model.generate_signals(lookback=4320, execute_threshold=1.2, close_threshold=0.5)
model.plot_pnl(test_start_date="2024-01-01")

In [ ]:
pnl_df = model.computePortfolioPnL(test_start_date="2024-01-01")
pnl_df

,timestamp,total_daily_pnl,total_daily_pnl_pct,total_pnl
0,2024-11-01 00:00:00,0.00,NaN,0.00
1,2024-11-01 00:05:00,0.00,0.000000,0.00
2,2024-11-01 00:10:00,0.00,0.000000,0.00
3,2024-11-01 00:15:00,0.00,0.000000,0.00
4,2024-11-01 00:20:00,0.00,0.000000,0.00
...,...,...,...,...
17276,2024-12-30 23:40:00,0.00,0.000000,9392.54
17277,2024-12-30 23:45:00,-30.50,-0.003247,9362.04
17278,2024-12-30 23:50:00,0.00,0.000000,9362.04
17279,2024-12-30 23:55:00,2.22,0.000237,9364.26


In [100]:
price_fetcher = BinancePriceFetcher(test_pairs)
# Fetch pair historical price
pair_portfolio_momentum = pair_portfolio[pair_portfolio["timestamp"] >= "2024-01-01"].reset_index(drop=True)

weights = np.array([1/len(test_pairs) for i in test_pairs])  # Equal-weighted

# Initialize strategy
strategy = SimpleStrategy(
    tickers=test_pairs,
    weights=weights,
    data=pair_portfolio_momentum
)

print(pair_portfolio.shape)
# Get momentum strategy returns
momentum_returns = strategy.momentum_strategy(lookback=30, hold_period=10)

(17281, 2)


# Comparison

In [101]:
returns = pd.DataFrame()
returns["Momentum"] = momentum_returns
returns["Pair_Trade"] = pnl_df[("total_daily_pnl_pct")]

port_metrics = PortfolioMetrics(returns.dropna())
summary = port_metrics.summary(risk_free_rate=0)
print(summary)

            Annualized Return  Annualized Volatility  Sharpe Ratio  \
Momentum             0.000000           0.000000e+00           NaN   
Pair_Trade           0.601255           7.257091e+10      0.120972   

            Max Drawdown Duration  Max Drawdown  Calmar Ratio    Skewness  \
Momentum                        1      0.000000           NaN         NaN   
Pair_Trade                      1     -1.681094      0.357657  131.213567   

                Kurtosis  VaR (5%)  CVaR (5%)  Tail Ratio   Omega Ratio  
Momentum             NaN -0.003319        NaN    1.050868           NaN  
Pair_Trade  17215.000058 -0.003319  -0.082878    1.050868  4.188392e+09  


In [96]:
wrc = WhiteRealityCheck(strategies=returns["Pair_Trade"], benchmark=returns["Momentum"])
wrc.run()
wrc.superior_strategies()

White Reality Check Results:
p-value: lower         0.163
consistent    0.163
upper         0.163
dtype: float64
Superior strategy indices:
[]


[]